In [9]:
!pip install pandas numpy scikit-learn tensorflow flask pymongo requests flask-cors scipy

In [10]:
import pandas as pd
import numpy as np
import pymongo
import requests
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model, load_model
from flask import Flask, request, jsonify
from flask_cors import CORS
from scipy.sparse import coo_matrix  # <-- For sparse matrix

In [12]:
# Load datasets
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')

print("Movies shape:", movies_df.shape)
print("Ratings shape:", ratings_df.shape)

Movies shape: (62423, 3)
Ratings shape: (25000095, 4)


In [13]:
# Clean data
movies_df.dropna(inplace=True)
ratings_df.dropna(inplace=True)

# Convert genres to list
movies_df['genres'] = movies_df['genres'].str.split('|')

print("Data preprocessing complete!")

Data preprocessing complete!


In [14]:
# Get unique user IDs and movie IDs
unique_users = ratings_df['userId'].unique()
unique_movies = ratings_df['movieId'].unique()

# Map IDs to indices (0-based)
user_id_to_index = dict(zip(unique_users, range(len(unique_users))))
movie_id_to_index = dict(zip(unique_movies, range(len(unique_movies))))

# Create rows, columns, and data for sparse matrix
rows = ratings_df['userId'].map(user_id_to_index).values
cols = ratings_df['movieId'].map(movie_id_to_index).values
data = ratings_df['rating'].values

# Build sparse matrix (COO format)
user_item_sparse = coo_matrix((data, (rows, cols)), 
                              shape=(len(unique_users), len(unique_movies)))

print("Sparse User-Item Matrix Shape:", user_item_sparse.shape)
print("Non-zero entries:", user_item_sparse.nnz)

Sparse User-Item Matrix Shape: (162541, 59047)
Non-zero entries: 25000095


In [18]:
# Convert sparse matrix to CSR (required for TruncatedSVD)
user_item_csr = user_item_sparse.tocsr()

# Apply Truncated SVD
svd = TruncatedSVD(n_components=10)  # 10 latent factors
user_latent = svd.fit_transform(user_item_csr)      # Dense matrix (users × components)
movie_latent = svd.components_.T                     # Dense matrix (movies × components)

print("✅ SVD completed!")
print("User Latent Vectors Shape:", user_latent.shape)
# ❌ DO NOT COMPUTE FULL SIMILARITY MATRIX (REMOVE THIS LINE):
# user_similarity = cosine_similarity(user_latent)   # <-- THIS CAUSED THE MEMORY ERROR

✅ SVD completed!
User Latent Vectors Shape: (162541, 10)


In [19]:
def recommend_movies(user_id, top_n=10):
    """Generate recommendations by computing similarity BETWEEN TARGET USER and ALL OTHERS on-the-fly."""
    if user_id not in user_id_to_index:
        return pd.DataFrame()  # User not found
    
    user_idx = user_id_to_index[user_id]
    
    # Get target user's latent vector
    target_vector = user_latent[user_idx]
    
    # ✅ Compute cosine similarity BETWEEN TARGET USER and EVERY OTHER USER
    # Dot product: (user_latent) × (target_vector).T → shape (num_users,)
    dot_products = user_latent @ target_vector  
    
    # Compute norms
    norm_target = np.linalg.norm(target_vector)
    norms = np.linalg.norm(user_latent, axis=1)  # Norm for every user
    
    # Avoid division by zero
    norms[norms == 0] = 1e-10
    
    # Compute similarities (O(n) operation – memory efficient)
    similarities = dot_products / (norm_target * norms)
    
    # Convert to list of (user_index, similarity)
    sim_scores = list(enumerate(similarities))
    
    # Sort by similarity (descending) and remove the target user itself
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = [score for idx, score in sim_scores if idx != user_idx]  # Exclude self
    
    # Get top-k similar users
    top_k_indices = [idx for idx, _ in sim_scores[:top_k]]
    
    # Get average ratings from top-k similar users
    sim_user_ratings = user_item_csr[top_k_indices].mean(axis=0)
    sim_user_ratings = np.array(sim_user_ratings).flatten()
    
    # Exclude movies already rated by the target user
    target_ratings = user_item_csr[user_idx].toarray().flatten()
    unrated_mask = (target_ratings == 0)
    sim_user_ratings = sim_user_ratings * unrated_mask  # Zero out already-rated movies
    
    # Get top-N movie indices
    top_indices = np.argpartition(sim_user_ratings, -top_n)[-top_n:]
    top_movie_ids = [unique_movies[i] for i in top_indices]
    
    # Fetch movie details
    recommendations = movies_df[movies_df['movieId'].isin(top_movie_ids)]
    return recommendations

# 🔍 Test the function (uncomment to try)
# recommendations = recommend_movies(user_id=1)
# recommendations

In [23]:
# Build the NCF model (same as before)
def build_ncf_model(num_users, num_movies, embedding_size=50):
    user_id_input = Input(shape=(1,), name='user_id')
    movie_id_input = Input(shape=(1,), name='movie_id')

    user_embedding = Embedding(input_dim=num_users, output_dim=embedding_size)(user_id_input)
    user_vec = Flatten()(user_embedding)

    movie_embedding = Embedding(input_dim=num_movies, output_dim=embedding_size)(movie_id_input)
    movie_vec = Flatten()(movie_embedding)

    product = Dot(axes=1, name='dot_product')([user_vec, movie_vec])
    merged = Concatenate()([user_vec, movie_vec])
    dense1 = Dense(128, activation='relu')(merged)
    dense2 = Dense(64, activation='relu')(dense1)
    output = Dense(1, activation='sigmoid')(dense2)

    model = Model(inputs=[user_id_input, movie_id_input], outputs=output)
    model.compile(loss='mse', optimizer='adam')
    return model

# 🔑 Sample a small subset for quick training (1% of data → ~250k ratings, trains in 5-15 min)
sample_frac = 0.01  # Change to 0.001 for even smaller/faster (0.1%)
ratings_df_sample = ratings_df.sample(frac=sample_frac, random_state=42)
print(f"✅ Sampled {len(ratings_df_sample)} ratings for quick training.")

# Get unique users/movies from the sample (smaller than full dataset)
unique_users_sample = ratings_df_sample['userId'].unique()
unique_movies_sample = ratings_df_sample['movieId'].unique()

# Create mappings: raw ID → index (0-based) for the sample
user_id_to_index = dict(zip(unique_users_sample, range(len(unique_users_sample))))
movie_id_to_index = dict(zip(unique_movies_sample, range(len(unique_movies_sample))))

# Map the sample to use indices
ratings_df_sample['user_idx'] = ratings_df_sample['userId'].map(user_id_to_index)
ratings_df_sample['movie_idx'] = ratings_df_sample['movieId'].map(movie_id_to_index)

# Filter valid rows (boolean indexing – no memory issues)
valid_rows = ratings_df_sample['user_idx'].notna() & ratings_df_sample['movie_idx'].notna()
ratings_df_sample = ratings_df_sample[valid_rows].copy()

# Get counts for embedding layers (based on sample)
num_users = len(unique_users_sample)
num_movies = len(unique_movies_sample)
print(f"✅ Unique users in sample: {num_users}, Unique movies: {num_movies}")

# Build and summarize model
ncf_model = build_ncf_model(num_users, num_movies)
ncf_model.summary()

# Prepare training data (USE MAPPED INDICES!)
X_train = {
    'user_id': ratings_df_sample['user_idx'].values.astype(int),
    'movie_id': ratings_df_sample['movie_idx'].values.astype(int)
}
y_train = ratings_df_sample['rating'].values

# Train the model (quick on small sample!)
history = ncf_model.fit(
    X_train, 
    y_train, 
    epochs=3,  # Reduced to 3 for speed (add more if needed)
    batch_size=128,  # Larger batch for faster training
    validation_split=0.2,
    verbose=1  # Shows progress
)

# Save the model
ncf_model.save('ncf_model_quick.h5')
print("✅ Quick NCF Model saved as ncf_model_quick.h5")

✅ Sampled 250001 ratings for quick training.
✅ Unique users in sample: 90126, Unique movies: 14306


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ user_id (InputLayer)          │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ movie_id (InputLayer)         │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_2 (Embedding)       │ (None, 1, 50)             │       4,506,300 │ user_id[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_3 (Embedding)       │ (None, 1, 50)             │         715,300 │ movie_id[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten_4 (Flatten)           │ (None, 50)                │               0 │ embedding_2[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten_5 (Flatten)           │ (None, 50)                │               0 │ embedding_3[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_2 (Concatenate)   │ (None, 100)               │               0 │ flatten_4[0][0],           │
│                               │                           │                 │ flatten_5[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_6 (Dense)               │ (None, 128)               │          12,928 │ concatenate_2[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_7 (Dense)               │ (None, 64)                │           8,256 │ dense_6[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_8 (Dense)               │ (None, 1)                 │              65 │ dense_7[0][0]              │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 5,242,849 (20.00 MB)

 Trainable params: 5,242,849 (20.00 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 63s 39ms/step - loss: 7.6110 - val_loss: 7.5551
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 58s 37ms/step - loss: 7.5530 - val_loss: 7.5555
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 57s 36ms/step - loss: 7.5528 - val_loss: 7.5556


✅ Quick NCF Model saved as ncf_model_quick.h5


In [24]:
# Predict a sample rating (use indices from the sample)
sample_user_idx = 0
sample_movie_idx = 0
predicted = ncf_model.predict([np.array([sample_user_idx]), np.array([sample_movie_idx])])
print(f"Predicted rating: {predicted[0][0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
Predicted rating: 0.9999998211860657


In [27]:
# In-memory "database" for user ratings - NO MongoDB required
user_ratings_store = {}

def save_user_ratings(user_id, ratings_dict):
    user_ratings_store[user_id] = ratings_dict
    print(f"✅ Ratings saved for User {user_id}")

def get_user_ratings(user_id):
    return user_ratings_store.get(user_id, {})

In [28]:
# Verify it works
save_user_ratings(1, {"Forrest Gump (1994)": 4.5, "Pulp Fiction (1994)": 5.0})
print("User 1 Ratings:", get_user_ratings(1))

✅ Ratings saved for User 1
User 1 Ratings: {'Forrest Gump (1994)': 4.5, 'Pulp Fiction (1994)': 5.0}


In [33]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
import pickle

# Step 1: Load and downsample data to a small, fast subset
movies_df = pd.read_csv('movies.csv', usecols=['movieId', 'title'])
ratings_df = pd.read_csv('ratings.csv', usecols=['userId', 'movieId', 'rating'])

# Keep only top 1000 active users and 500 popular movies
top_users = ratings_df['userId'].value_counts().head(1000).index
ratings_small = ratings_df[ratings_df['userId'].isin(top_users)]

top_movies = ratings_small['movieId'].value_counts().head(500).index
ratings_small = ratings_small[ratings_small['movieId'].isin(top_movies)]

# Merge with movie titles
ratings_small = ratings_small.merge(movies_df, on='movieId')

# Step 2: Create user-item matrix
user_item_matrix = ratings_small.pivot_table(
    index='userId', 
    columns='title', 
    values='rating'
).fillna(0)

# Step 3: Train KNN model (takes ~10 seconds)
knn_model = NearestNeighbors(metric='cosine', n_neighbors=10, n_jobs=-1)
knn_model.fit(user_item_matrix)

# Step 4: Save the files app.py requires
with open("submission_recommender_model.pkl", "wb") as f:
    pickle.dump(knn_model, f)

with open("submission_user_item_matrix.pkl", "wb") as f:
    pickle.dump(user_item_matrix, f)

print("✅ Success! Both required files saved:")
print("- submission_recommender_model.pkl")
print("- submission_user_item_matrix.pkl")

✅ Success! Both required files saved:
- submission_recommender_model.pkl
- submission_user_item_matrix.pkl
